In [1]:
# import libraries
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import re
import germansentiment

In [5]:
# define stopwords
stop_words = open('stopwords.txt','r').read().split('\n')

In [6]:
import csv

# import scraped data
d1 = pd.read_csv("Product_Hunt.csv")
d1 = d1['Review text']

d2 = pd.read_csv("Trustpilot.csv")
d2 = d2['Review text']

d = [d1, d2]
d = pd.concat(d)

In [7]:
import gensim
from gensim.utils import simple_preprocess

# topic modeling
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

data = d.values.tolist()
data_words = list(sent_to_words(data))

# remove stop words
data_words = remove_stopwords(data_words)

In [8]:
import gensim.corpora as corpora

# create dictionary
id2word = corpora.Dictionary(data_words)

# create corpus
texts = data_words

# term document frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [9]:
from pprint import pprint

# define number of topics
num_topics = 5

# build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

# print the keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.010*"great" + 0.006*"best" + 0.006*"ideas" + 0.006*"better" + '
  '0.005*"experience" + 0.005*"good" + 0.005*"awesome" + 0.004*"money" + '
  '0.004*"got" + 0.004*"service"'),
 (1,
  '0.019*"great" + 0.007*"work" + 0.005*"support" + 0.005*"help" + '
  '0.005*"access" + 0.005*"helpful" + 0.004*"useful" + 0.004*"new" + '
  '0.004*"open" + 0.004*"subscription"'),
 (2,
  '0.006*"like" + 0.005*"free" + 0.004*"new" + 0.004*"support" + '
  '0.004*"everything" + 0.004*"world" + 0.003*"work" + 0.003*"best" + '
  '0.003*"helpful" + 0.003*"could"'),
 (3,
  '0.025*"good" + 0.013*"amazing" + 0.008*"like" + 0.006*"service" + '
  '0.006*"nice" + 0.005*"love" + 0.005*"future" + 0.004*"work" + 0.003*"cool" '
  '+ 0.003*"help"'),
 (4,
  '0.009*"like" + 0.005*"support" + 0.005*"open" + 0.005*"help" + '
  '0.004*"useful" + 0.004*"know" + 0.004*"amazing" + 0.004*"development" + '
  '0.004*"research" + 0.003*"dall"')]


In [10]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis
import os

# output graphic representation
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
pyLDAvis.save_html(LDAvis_prepared, "LDA_Reviews_EN_"+str(num_topics) +'.html')
LDAvis_prepared

"# Visualize the topics\npyLDAvis.enable_notebook()\nLDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))\n# # this is a bit time consuming - make the if statement True\n# # if you want to execute visualization prep yourself\nif 1 == 1:\n    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)\n    with open(LDAvis_data_filepath, 'wb') as f:\n        pickle.dump(LDAvis_prepared, f)\n# load the pre-prepared pyLDAvis data from disk\nwith open(LDAvis_data_filepath, 'rb') as f:\n    LDAvis_prepared = pickle.load(f)\npyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')\nLDAvis_prepared\n"